In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_bureau_feats
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy
from sklearn.metrics import f1_score

In [2]:
import sys
sys.path.append('ml_lib/')
from encoding import FreqeuncyEncoding
from custom_classifier_mutliclass import Estimator
from hyperopt_multiclass import HyperOptModelSelection
from hyperopt import hp
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

In [3]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [4]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')

In [5]:
target_map = {'No Top-up Service': 0,
 '12-18 Months': 1,
 '18-24 Months': 2,
 '24-30 Months': 3,
 '30-36 Months': 4,
 '36-48 Months': 5,
 ' > 48 Months': 6,
 }
train['Top-up Month'] = train['Top-up Month'].map(target_map)

In [6]:
reverse_map = {v:k for k,v in target_map.items()}

In [7]:
reverse_map

{0: 'No Top-up Service',
 1: '12-18 Months',
 2: '18-24 Months',
 3: '24-30 Months',
 4: '30-36 Months',
 5: '36-48 Months',
 6: ' > 48 Months'}

In [8]:
train['Top-up Month'].value_counts()

0    106677
6      8366
5      3656
3      3492
4      3062
2      2368
1      1034
Name: Top-up Month, dtype: int64

In [9]:
df = pd.read_pickle('training_data/data.pkl')
df2 = pd.read_pickle('tfidf_feats1.pkl')
df = df.merge(df2,on='ID',how='left')
del df2
import gc
gc.collect()

0

In [10]:
df.shape

(143400, 1514)

In [11]:
# df.shape

In [12]:
cat_cols=[]
target = 'Top-up Month'
drop_cols = ['ID','DisbursalDate','MaturityDAte','AuthDate','AssetID','PiNCODE']
num_cols = (df.columns[~df.columns.isin([target]+drop_cols+cat_cols)].tolist())
use_cols = cat_cols+num_cols

In [13]:
len(num_cols)

1512

In [14]:
# df[pd.read_pickle('training_data/500_feats_data_tfidf.pkl').columns].to_pickle('training_data/500_feats_data_tfidf.pkl')

In [15]:
train = df[df.ID.isin(train.ID)]
test = df[df.ID.isin(test.ID)]
train = train.sort_values('ID',ascending=True)
test = test.sort_values('ID',ascending=True)
train[target] = train[target].astype('int')

In [16]:
train.shape,test.shape

((128655, 1514), (14745, 1514))

In [17]:
del df,
gc.collect()

44

In [18]:
folds = StratifiedKFold(5, shuffle = True, random_state = 2)
folds = [(x,y) for x,y in folds.split(train,train[target])]

In [19]:
len(use_cols)

1512

In [20]:
lgbm_space = {
    "n_estimators": 5000,
    "num_leaves": hp.quniform("num_leaves", 32, 192, 32),
    "min_child_weight": hp.quniform("min_child_weight", 10, 100, 20),
    "subsample": hp.quniform("subsample", 0.5, 1, 0.1),
    "colsample_bytree": hp.quniform("colsample_bytree", 0.5, 1, 0.1),
    "subsample_freq": 5,
    "objective": "multiclass",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": -1,
}

In [21]:
est = Estimator(model = LGBMClassifier(),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [23]:
# %%time
# hyp= HyperOptModelSelection(model=est,space=lgbm_space,max_evals=50,is_maximize=True,log_file_path='hyp4.txt')

In [24]:
%%time
# hyp.fit(train[use_cols].values,train[target].values)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.81 µs


In [25]:
# params={'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 40.0, 'n_estimators': 5000,
#         'n_jobs': -1, 'num_leaves': 96, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

params = {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 20.0, 'n_estimators': 5000,
 'n_jobs': -1, 'num_leaves': 64, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

In [26]:
# est = Estimator(model = LGBMClassifier(colsample_bytree=0.7000000000000001, min_child_weight=60.0,learning_rate=0.1,
#                n_estimators=5000, num_leaves=64, objective='multiclass',metric='None',
#                subsample_freq=5),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

est = Estimator(model = LGBMClassifier(**params),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [27]:
temp = est.fit_transform(train[use_cols].values,train[target].values)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.277424	valid_1's multi_logloss: 0.147215
Early stopping, best iteration is:
[93]	valid_0's multi_logloss: 0.277288	valid_1's multi_logloss: 0.153245
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.273074	valid_1's multi_logloss: 0.148219
[200]	valid_0's multi_logloss: 0.278714	valid_1's multi_logloss: 0.0883823
Early stopping, best iteration is:
[106]	valid_0's multi_logloss: 0.272919	valid_1's multi_logloss: 0.143239
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.273832	valid_1's multi_logloss: 0.147946
Early stopping, best iteration is:
[89]	valid_0's multi_logloss: 0.273612	valid_1's multi_logloss: 0.157696
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.274118	valid_1's multi_logloss: 0.148404
[200]	valid_0's multi_logloss: 0.279768	valid_1's multi_

In [29]:
np.mean([0.277288,0.272919,0.273612,0.274056,0.264861])

0.27254720000000004

In [28]:
est.cv_scores,est.overall_cv_score

([0.7150687527492691,
  0.7253377493324094,
  0.7226694619410213,
  0.7227856518915358,
  0.7314023595137764],
 0.7236004766926883)

In [35]:
%%time
lgbm_space = {
    "n_estimators": 5000,
    "num_leaves": hp.quniform("num_leaves", 4, 16, 4),
    "min_child_weight": hp.quniform("min_child_weight", 10, 100, 20),
    "subsample": hp.quniform("subsample", 0.5, 1, 0.1),
    "colsample_bytree": hp.quniform("colsample_bytree", 0.5, 1, 0.1),
    "subsample_freq": 5,
    "objective": "multiclass",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": -1,
}
hyp= HyperOptModelSelection(model=est,space=lgbm_space,max_evals=50,is_maximize=True,log_file_path='hyp_meta.txt')

CPU times: user 798 µs, sys: 2 µs, total: 800 µs
Wall time: 548 µs


In [67]:
# hyp.fit(temp,train[target].values)

In [37]:
meta_est = hyp.best_estimator

In [38]:
meta_est.model

LGBMClassifier(colsample_bytree=0.5, min_child_weight=80.0, n_estimators=5000,
               num_leaves=12, objective='multiclass', subsample=0.8,
               subsample_freq=5)

In [70]:
meta_params = {'boosting_type': 'gbdt',
 'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'min_child_weight': 80.0,
 'n_estimators': 5000,
 'n_jobs': -1,
 'num_leaves': 12,
 'objective': 'multiclass',
 'subsample': 0.8,
 'subsample_freq': 5}

meta_est = Estimator(model = LGBMClassifier(**meta_params),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [71]:
oof_preds = meta_est.fit_transform(temp,train[target].values)
meta_est.cv_scores,meta_est.overall_cv_score

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.278479	valid_1's multi_logloss: 0.259084
[200]	valid_0's multi_logloss: 0.278844	valid_1's multi_logloss: 0.25074
Early stopping, best iteration is:
[127]	valid_0's multi_logloss: 0.278236	valid_1's multi_logloss: 0.256277
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.271899	valid_1's multi_logloss: 0.260577
[200]	valid_0's multi_logloss: 0.272413	valid_1's multi_logloss: 0.252039
Early stopping, best iteration is:
[125]	valid_0's multi_logloss: 0.271612	valid_1's multi_logloss: 0.257948
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.273313	valid_1's multi_logloss: 0.260081
[200]	valid_0's multi_logloss: 0.273264	valid_1's multi_logloss: 0.251644
Early stopping, best iteration is:
[124]	valid_0's multi_logloss: 0.272774	valid_1's multi_logloss: 0.257486
Training until validation scores don't impro

([0.7189437536877117,
  0.732607017588078,
  0.7224522777567273,
  0.7259197637012166,
  0.7361811736390812],
 0.7272938602752097)

In [73]:
np.mean([0.262225,0.273189,0.272774,0.271612,0.278236])

0.2716072

In [41]:
pd.Series(oof_preds.argmax(axis=1)).value_counts(1)

0    0.882857
6    0.047204
5    0.018429
3    0.016463
4    0.015110
2    0.013089
1    0.006848
dtype: float64

In [40]:
# feat_imp = est.feature_importance_df(use_cols)
# train[feat_imp.column.iloc[:500].tolist()+['ID',target]].to_pickle('training_data/500_feats_data_tfidf.pkl')

In [42]:
f1_score(train[target],pd.Series(oof_preds.argmax(axis=1)),average='macro')

0.7272938602752097

In [50]:
test_preds = [est.predict_proba(test[use_cols].values) for est in est.fitted_models]
meta_test_df = np.mean(test_preds,axis=0)

In [56]:
test_preds = [est.predict_proba(meta_test_df) for est in meta_est.fitted_models]

In [61]:
pd.Series(meta_test_df.argmax(1)).value_counts(1)

0    0.918616
6    0.037369
5    0.017904
4    0.007731
2    0.006850
3    0.006104
1    0.005426
dtype: float64

In [47]:
# pd.Series(np.sum(test_preds,axis=0).argmax(axis=1)).value_counts(1)

In [57]:
test['Top-up Month'] = np.sum(test_preds,axis=0).argmax(axis=1)

In [58]:
test['Top-up Month'].value_counts(1)

0    0.912648
6    0.038047
5    0.018379
4    0.010173
2    0.007664
3    0.006985
1    0.006104
Name: Top-up Month, dtype: float64

In [63]:
sub = test[['ID','Top-up Month']]

In [65]:
sub['Top-up Month'] = sub['Top-up Month'].map(reverse_map)

In [66]:
sub.to_csv('LGBM_TFIDF_META_CV0.7273.csv',index=False)